In [98]:
import folium 
import pandas as pd
import geopandas as gpd 
from shapely.geometry import Point

In [99]:
# 道路情報から必要のないものをフィルタリングするためのリスト
FLTER_ROAD_MASK = {
    "trunk_link",
    "tertiary",
    "motorway",
    "motorway_link",
    "steps",
    None,
    " ",
    "pedestrian",
    "primary",
    "primary_link",
    "footway",
    "tertiary_link",
    "trunk",
    "secondary",
    "secondary_link",
    "tertiary_link",
    "bridleway",
    "service",
    "footway",
    "unknown"
}

# ポイントを生成する間隔（メートル）
MINI_DIST = 200

In [100]:
# 関東のポリゴンを取得
tokyo_polygon = gpd.read_file("/workspace/app/Green-View-Index/tokyo-area/N03-20240101_13.shp")
# 渋谷区のポリゴンを抽出
shibuya_polygon = tokyo_polygon[tokyo_polygon["N03_007"] == "13113"]
# ポリゴンを結合
shibuya_polygon = shibuya_polygon.unary_union

/tmp/ipykernel_2113/1879520858.py:6: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  shibuya_polygon = shibuya_polygon.unary_union


In [101]:
# 道路情報の取得
roads = gpd.read_file("/workspace/app/Green-View-Index/kanto-latest-free.shp/gis_osm_roads_free_1.shp")

# ポリゴン内の道路のみを抽出
roads_in_polygon = roads[roads.within(shibuya_polygon)]

# 不要な道路を除外する
roads_filtered = roads_in_polygon[~roads_in_polygon["fclass"].isin(FLTER_ROAD_MASK)]

# 座標系が緯度経度WGS84 (EPSG:4326)の場合、Webメルカトル（メートル）EPSG:3857に変換
if roads_filtered.crs != "EPSG:3857":
    roads_filtered = roads_filtered.to_crs(epsg=3857)

In [102]:
from shapely.geometry import Point

# mini_dist: 点を生成する間隔（メートル）
mini_dist = MINI_DIST # 20メートルごとに点を生成

# 生成した点を格納するためのリスト
points = []

# フィルタリングされた道路データ（EPSG:3857で既に変換済み）を使用
for line in roads_filtered.geometry:
    # ラインストリングが存在し、長さがあるか確認
    if line and line.length > 0:
        # ラインストリングの長さを取得（メートル単位）
        line_length = line.length

        # 指定された距離ごとに点を生成
        for distance in range(0, int(line_length), mini_dist):
            point = line.interpolate(distance)  # 指定されたメートル単位の位置に点を生成
            points.append(point)

# 生成されたポイントを確認
print(f"生成されたポイント数: {len(points)}")
print(points[:5])

生成されたポイント数: 3102
[<POINT (15551222.969 4253076.781)>, <POINT (15551280.755 4252970.155)>, <POINT (15551286.925 4253130.481)>, <POINT (15551234.078 4253311.101)>, <POINT (15551128.648 4253249.608)>]


In [103]:
# WGS84 (EPSG:4326) に戻す
gdf_points = gpd.GeoDataFrame(geometry=points, crs="EPSG:3857")
gdf_points_wgs84 = gdf_points.to_crs(epsg=4326)

# WGS84座標系でのポイントを確認
print(gdf_points_wgs84.geometry[:5])

0    POINT (139.69901 35.65371)
1    POINT (139.69953 35.65293)
2     POINT (139.69959 35.6541)
3    POINT (139.69911 35.65542)
4    POINT (139.69817 35.65497)
Name: geometry, dtype: geometry


In [104]:
# 地図の作成
map = folium.Map(location=[35.658034, 139.701636], zoom_start=15, control_scale=True)

# GeoDataFrameの geometry 列を反復処理してポイントを追加
for point in gdf_points_wgs84.geometry:
    folium.CircleMarker(
        location=[point.y, point.x],  # 緯度（y）、経度（x）
        radius=1,
        color="red",
        fill=True,
        fill_color="red",
    ).add_to(map)

# 地図を表示
map